# Panle Regression - Firm Characteristics

### Random Effects Panel Regression

In [11]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

Data

In [12]:
df = pd.read_csv("Dataframes/characteristics.csv")
X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,1.532200e+09,4.413001e+09,3.426001e+09,0.799347,10.637479,207295.0,-1
1,AVY.N,2013-04-01,Materials,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1
2,AVY.N,2013-07-01,Materials,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
3,AVY.N,2013-10-01,Materials,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
4,AVY.N,2014-01-01,Materials,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
19335,POOL.OQ,2021-10-01,Retailing,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
19336,POOL.OQ,2022-01-01,Retailing,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
19337,POOL.OQ,2022-04-01,Retailing,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
19338,POOL.OQ,2022-07-01,Retailing,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1


In [13]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [14]:
#removing outliers
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_clean = df.copy()
df_clean["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_clean = df_clean.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_clean = df_clean.dropna()
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,1.532200e+09,4.413001e+09,3.426001e+09,0.799347,10.637479,207295.0,-1
1,AVY.N,2013-04-01,Materials,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1
2,AVY.N,2013-07-01,Materials,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
3,AVY.N,2013-10-01,Materials,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
4,AVY.N,2014-01-01,Materials,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
19335,POOL.OQ,2021-10-01,Retailing,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
19336,POOL.OQ,2022-01-01,Retailing,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
19337,POOL.OQ,2022-04-01,Retailing,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
19338,POOL.OQ,2022-07-01,Retailing,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1


Standardising

In [15]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Enterprise Value"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = MinMaxScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,0.065123,0.027862,0.001166,0.027195,0.298034,0.001183,-1
1,AVY.N,2013-04-01,Materials,2.482,0.064917,0.028207,0.001471,0.027308,0.340793,0.000000,-1
2,AVY.N,2013-07-01,Materials,1.068,0.065247,0.028199,0.001453,0.027321,0.264198,0.000000,-1
3,AVY.N,2013-10-01,Materials,8.095,0.064955,0.028065,0.001459,0.027317,0.272179,0.001348,-1
4,AVY.N,2014-01-01,Materials,1.471,0.065443,0.028238,0.001667,0.027338,0.315017,0.000900,-1
...,...,...,...,...,...,...,...,...,...,...,...
19335,POOL.OQ,2021-10-01,Retailing,17.194,0.064377,0.032245,0.006004,0.030598,0.248134,0.000900,-1
19336,POOL.OQ,2022-01-01,Retailing,40.267,0.062055,0.034274,0.007826,0.031566,0.360861,0.000470,-1
19337,POOL.OQ,2022-04-01,Retailing,34.342,0.064385,0.032488,0.005848,0.030290,0.184744,0.001660,-1
19338,POOL.OQ,2022-07-01,Retailing,1.503,0.068358,0.031540,0.004842,0.029495,0.195817,0.001059,-1


In [16]:
df_panel = df_clean.copy()
df_panel

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,0.065123,0.027862,0.001166,0.027195,0.298034,0.001183,-1
1,AVY.N,2013-04-01,Materials,2.482,0.064917,0.028207,0.001471,0.027308,0.340793,0.000000,-1
2,AVY.N,2013-07-01,Materials,1.068,0.065247,0.028199,0.001453,0.027321,0.264198,0.000000,-1
3,AVY.N,2013-10-01,Materials,8.095,0.064955,0.028065,0.001459,0.027317,0.272179,0.001348,-1
4,AVY.N,2014-01-01,Materials,1.471,0.065443,0.028238,0.001667,0.027338,0.315017,0.000900,-1
...,...,...,...,...,...,...,...,...,...,...,...
19335,POOL.OQ,2021-10-01,Retailing,17.194,0.064377,0.032245,0.006004,0.030598,0.248134,0.000900,-1
19336,POOL.OQ,2022-01-01,Retailing,40.267,0.062055,0.034274,0.007826,0.031566,0.360861,0.000470,-1
19337,POOL.OQ,2022-04-01,Retailing,34.342,0.064385,0.032488,0.005848,0.030290,0.184744,0.001660,-1
19338,POOL.OQ,2022-07-01,Retailing,1.503,0.068358,0.031540,0.004842,0.029495,0.195817,0.001059,-1


#### Random Effects

In [17]:
# turn dataframe into right dimension for panel regression with multiindex
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)

# define the dependent variable
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise"]
X = df_clean[X_var_names]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# perform the random effects panel regression
rfc = RandomForestRegressor(n_estimators=200)
rfc.fit(X_train, y_train)


TypeError: TimeSeriesSplit.__init__() takes from 1 to 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

In [8]:
df_clean

GICS Industry Group Name  \
Instrument Date                                  
AVY.N      2013-01-01                Materials   
           2013-04-01                Materials   
           2013-07-01                Materials   
           2013-10-01                Materials   
           2014-01-01                Materials   
...                                        ...   
POOL.OQ    2021-10-01                Retailing   
           2022-01-01                Retailing   
           2022-04-01                Retailing   
           2022-07-01                Retailing   
           2022-10-01                Retailing   

                       Earnings Per Share - Actual Surprise  Revenue - Actual  \
Instrument Date                                                                 
AVY.N      2013-01-01                                11.178          0.065123   
           2013-04-01                                 2.482          0.064917   
           2013-07-01                                 1.068          0.065247   
           2013-10-01                                 8.095          0.064955   
           2014-01-01                                 1.471          0.065443   
...                                                     ...               ...   
POOL.OQ    2021-10-01                                17.194          0.064377   
           2022-01-01                                40.267          0.062055   
           2022-04-01                                34.342          0.064385   
           2022-07-01                                 1.503          0.068358   
           2022-10-01                                 4.196          0.065637   

                       Enterprise Value  Market Capitalization  \
Instrument Date                                                  
AVY.N      2013-01-01          0.027862               0.001166   
           2013-04-01          0.028207               0.001471   
           2013-07-01          0.028199               0.001453   
           2013-10-01          0.028065               0.001459   
           2014-01-01          0.028238               0.001667   
...                                 ...                    ...   
POOL.OQ    2021-10-01          0.032245               0.006004   
           2022-01-01          0.034274               0.007826   
           2022-04-01          0.032488               0.005848   
           2022-07-01          0.031540               0.004842   
           2022-10-01          0.031045               0.004337   

                       Enterprise Value To Sales (Daily Time Series Ratio)  \
Instrument Date                                                              
AVY.N      2013-01-01                                           0.027195     
           2013-04-01                                           0.027308     
           2013-07-01                                           0.027321     
           2013-10-01                                           0.027317     
           2014-01-01                                           0.027338     
...                                                                  ...     
POOL.OQ    2021-10-01                                           0.030598     
           2022-01-01                                           0.031566     
           2022-04-01                                           0.030290     
           2022-07-01                                           0.029495     
           2022-10-01                                           0.029064     

                       3 Month Total Return    Volume  loss firm status  
Instrument Date                                                          
AVY.N      2013-01-01              0.298034  0.001183                -1  
           2013-04-01              0.340793  0.000000                -1  
           2013-07-01              0.264198  0.000000                -1  
           2013-10-01              0.272179  0.001348                -1  

#### Fixed Effects

In [9]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = rfc.predict(X_test)



In [10]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean squared error: ", mse)
print("R-squared: ", r2)


Mean squared error:  282.27897992507695
R-squared:  -0.011513906262198725
